In [ ]:
import re
import os
import math
import torch
import pandas as pd
import pytorch_lightning as pl

from tqdm import tqdm
from rouge import Rouge
from datetime import datetime

from torch import nn
from konlpy.tag import Mecab
from tensorboardX import SummaryWriter
from torch.nn import functional as F
from transformers import AutoTokenizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model_id = "psyche/KoT5-summarization"
train_df = pd.read_csv('../dataset/cleaned_train.csv')
val_df = pd.read_csv('../dataset/cleaned_dev.csv')
test_df = pd.read_csv("../dataset/test.csv")

epochs = 100
batch_size = 16
num_workers = 0
patience = 5

init_lr = 0.0001
max_lr = 0.001
warmup_epochs = 10
T_0 = 100
T_mult = 1
T_gamma = 0.5

dig_max_len = 1024
sum_max_len = 512

tokenizer = AutoTokenizer.from_pretrained(model_id)
special_tokens_dict={'additional_special_tokens': ['#Person1#', '#Person2#','#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', 
                                                   '#Address#', '#PassportNumber#', '#CardNumber#', '#Email#', '#DateOfBirth#',
                                                   '<sep>']}

tokenizer.add_special_tokens(special_tokens_dict)
print(tokenizer.special_tokens_map)

remove_tokens = [
    '<usr>',
    f"{tokenizer.unk_token}", 
    f"{tokenizer.eos_token}", 
    f"{tokenizer.pad_token}"
]

{'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', '#Address#', '#PassportNumber#', '#CardNumber#', '#Email#', '#DateOfBirth#', '<sep>']}


/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, input_len, summ_len, is_train=True):
        self.tokenizer = tokenizer
        self.df = df.copy()
        self.source_len = input_len
        self.summ_len = summ_len
        self.is_train = is_train

        self.df.loc[:, 'dialogue'] = self.df['dialogue'].apply(self.add_sep_tokens)

        if self.is_train:
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), return_tensors="pt", padding=True,
                                add_special_tokens=True, truncation=True, max_length=512, return_token_type_ids=False).input_ids
            self.labels = tokenizer(self.df['summary'].tolist(), return_tensors="pt", padding=True,
                                add_special_tokens=True, truncation=True, max_length=100, return_token_type_ids=False).input_ids
        else:
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), return_tensors="pt", padding=True,
                                add_special_tokens=True, truncation=True, max_length=512, return_token_type_ids=False).input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.is_train:
            return self.input_ids[idx], self.labels[idx]
        else:
            return self.input_ids[idx]

    def add_sep_tokens(self, dialogue):
        pattern = r'(#Person\d+#)'
        parts = re.split(pattern, dialogue)
        result = []
        prev_speaker = None
        for part in parts:
            if re.match(pattern, part):
                if prev_speaker and prev_speaker != part:
                    result.append('<sep>')
                prev_speaker = part
            result.append(part)
        return ''.join(result)


In [ ]:
train_dataset = CustomDataset(train_df[['dialogue', 'summary']], tokenizer, dig_max_len, sum_max_len)
val_dataset = CustomDataset(val_df[['dialogue', 'summary']], tokenizer, dig_max_len, sum_max_len)
test_dataset = CustomDataset(test_df[['dialogue']], tokenizer, dig_max_len, sum_max_len, is_train=False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
def ids_to_words(tokenizer, preds, labels):
    decoded_preds = tokenizer.batch_decode(preds, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()

    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]
        
    return replaced_predictions, replaced_labels

In [ ]:
def compute_metrics(replaced_predictions, replaced_labels):
    rouge = Rouge()

    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)
    result = {key: value["f"] for key, value in results.items()}
    
    return result

In [ ]:
class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [ ]:
def sample_sequence(model, input_ids, max_length):
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        do_sample=True,
        top_k=0,
        temperature=0.7,
        no_repeat_ngram_size=2
    )
    return output

def greedy_sequence(model, input_ids, max_length):
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        num_beams=1,
        do_sample=False
    )
    return output

def compute_rouge(predictions, references):
    rouge = Rouge()
    scores = rouge.get_scores(predictions, references, avg=True)
    return {key: value['f'] for key, value in scores.items()}

In [ ]:
def scst_loss(pred_ids, baseline_ids, labels, tokenizer, model):
    replaced_preds, replaced_labels = ids_to_words(tokenizer, pred_ids, labels)
    replaced_baseline, _ = ids_to_words(tokenizer, baseline_ids, labels)

    pred_scores = compute_rouge(replaced_preds, replaced_labels)
    baseline_scores = compute_rouge(replaced_baseline, replaced_labels)

    rewards = torch.tensor([pred_scores['rouge-l'] for _ in range(len(pred_ids))]).to(pred_ids.device)
    baseline = torch.tensor([baseline_scores['rouge-l'] for _ in range(len(baseline_ids))]).to(baseline_ids.device)

    log_probs = compute_log_probs(model, pred_ids, labels)
    loss = -((rewards - baseline) * log_probs).mean()

    return loss

def compute_log_probs(model, pred_ids, labels):
    with torch.no_grad():
        outputs = model(input_ids=pred_ids, labels=labels)
        logits = outputs.logits
    
    log_probs = F.log_softmax(logits, dim=-1)
    token_log_probs = log_probs.gather(2, labels.unsqueeze(2)).squeeze(2)
    mask = (labels != tokenizer.pad_token_id).float()
    token_log_probs = token_log_probs * mask
    seq_log_probs = token_log_probs.sum(dim=1)
    
    return seq_log_probs


In [ ]:
def train_scst(epoch, model, device, train_loader, optimizer, train_step, writer):
    model.train()
    total_loss = 0.0

    for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Training Epoch {epoch}", leave=False):
        input_ids = batch[0].to(device, dtype=torch.long)
        labels = batch[1].to(device, dtype=torch.long)

        optimizer.zero_grad()

        baseline_ids = greedy_sequence(model, input_ids, max_length=256)
        pred_ids = sample_sequence(model, input_ids, max_length=256)

        loss = scst_loss(pred_ids, baseline_ids, labels, tokenizer, model)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        train_step += 1

    avg_loss = total_loss / len(train_loader)
    writer.add_scalar('Loss/train', avg_loss, epoch)
    
    return train_step, avg_loss

In [ ]:
def validate_scst(tokenizer, model, device, val_loader, writer, epoch):
    model.eval()
    total_loss = 0
    all_results = []
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for idx, batch in tqdm(enumerate(val_loader), total=len(val_loader), desc="Validating", leave=False):
            input_ids = batch[0].to(device, dtype=torch.long)
            labels = batch[1].to(device, dtype=torch.long)

            pred_ids = model.generate(input_ids=input_ids, max_length=256, num_beams=4)

            replaced_predictions, replaced_labels = ids_to_words(tokenizer, pred_ids, labels)
            result = compute_metrics(replaced_predictions, replaced_labels)

            all_results.append(result)
            all_predictions.extend(replaced_predictions)
            all_labels.extend(replaced_labels)

    avg_result = {key: sum(r[key] for r in all_results) / len(all_results) for key in all_results[0]}
    writer.add_scalar('ROUGE/rouge-1', avg_result['rouge-1'], epoch)
    writer.add_scalar('ROUGE/rouge-2', avg_result['rouge-2'], epoch)
    writer.add_scalar('ROUGE/rouge-l', avg_result['rouge-l'], epoch)

    return avg_result, all_predictions, all_labels

In [ ]:
train_step = 0

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
save_path = os.path.join("./T5_runs", timestamp)

weights_path = os.path.join(save_path, 'weights')
logs_path = os.path.join(save_path, 'logs')
os.makedirs(weights_path, exist_ok=True)
os.makedirs(logs_path, exist_ok=True)

best_avg_rouge = 0
early_stopping_counter = 0
writer = SummaryWriter(log_dir=logs_path)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=T_0, T_mult=T_mult, eta_max=max_lr,  T_up=warmup_epochs, gamma=T_gamma)

for epoch in range(1, epochs + 1):
    train_step, train_loss = train_scst(epoch, model, device, train_loader, optimizer, train_step, writer)
    val_result, val_predictions, val_labels = validate_scst(tokenizer, model, device, val_loader, writer, epoch)

    avg_rouge = (val_result['rouge-1'] + val_result['rouge-2'] + val_result['rouge-l']) / 3
    print(f"Epoch {epoch}/{epochs}")
    print(f"Train Loss: {train_loss:.6f}")
    print(f"Rouge-1: {val_result['rouge-1']:.6f}, Rouge-2: {val_result['rouge-2']:.6f}, Rouge-l: {val_result['rouge-l']:.6f}")
    print(f"Average ROUGE: {avg_rouge:.6f}")

    print('-'*150)
    for i in range(3):
        print(f"PRED: {val_predictions[i]}")
        print(f"GOLD: {val_labels[i]}")
        print('-'*150)

    if avg_rouge > best_avg_rouge:
        best_avg_rouge = avg_rouge
        early_stopping_counter = 0
        torch.save(model.state_dict(), os.path.join(weights_path, 'best.pth'))
        print(f"New best model saved with average ROUGE: {best_avg_rouge:.6f}")

    else:
        early_stopping_counter += 1
        print(f"Not improved. Early stopping counter: {early_stopping_counter}/{patience}")

    if early_stopping_counter >= patience:
        print("Early stopping triggered.")
        break

    torch.save(model.state_dict(), os.path.join(weights_path, f'epoch-{epoch}.pth'))
    print()

writer.close()
torch.save(model.state_dict(), os.path.join(weights_path, 'last.pth'))
print("Training completed. Last model saved.")


Epoch 1/100
Train Loss: 0.765012
Validation Loss: 0.586895, Rouge-1: 0.342694, Rouge-2: 0.111150, Rouge-l: 0.320415
Average ROUGE: 0.258086
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 숨쉬기가 힘들다고 느끼고, 알레르기가 있는지 묻습니다. #Person2# 는 폐 전문의에게 천식에 대한 검사를 받게 할 것입니다.                                     
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   헤이, 지미는 3시 30분에 운동하러 가자고 제안한다. 그들은 단지 두 날을 바꾸는 것을 제안하고, 금요일에 다리를 할 수 있다고 말한다.                                   
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
--------------------------

Epoch 2/100
Train Loss: 0.580652
Validation Loss: 0.557176, Rouge-1: 0.338138, Rouge-2: 0.108483, Rouge-l: 0.318480
Average ROUGE: 0.255034
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   의사 선생님은 #Person1# 에게 최근 숨쉬기가 힘들다고 말하고, 알레르기가 있는지 묻습니다. #Person2# 는 폐 전문의에게 천식에 대한 검사를 받도록 할 것입니다.                         
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   헤이, 지미는 3시 30분에 헬스장에 가자고 제안한다. 그들은 금요일에 두 날을 바꾸기로 결정하고 다시 만나기로 한다.                                
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
--------------------------------------

Epoch 3/100
Train Loss: 0.530246
Validation Loss: 0.534735, Rouge-1: 0.373175, Rouge-2: 0.130584, Rouge-l: 0.350167
Average ROUGE: 0.284642
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 #Person2# 에게 최근 숨쉬기가 힘들다고 말합니다. #Person1# 는 폐 전문의에게 천식에 대한 검사를 받도록 요청합니다.                                 
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   지미는 #Person1# 에게 운동하러 가자고 제안하지만, #Person2# 은 지미 때문에 모든 것이 망가지고 있다고 말한다.                                    
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
----------------------------------

Epoch 4/100
Train Loss: 0.484493
Validation Loss: 0.526318, Rouge-1: 0.363181, Rouge-2: 0.125128, Rouge-l: 0.341818
Average ROUGE: 0.276709
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 숨쉬기가 힘들다고 #Person2# 에게 말하고, 알레르기는 없지만 운동을 할 때 많이 나타난다고 말한다. #Person1# 는 폐 전문의에게 천식 검사를 받도록 요청한다.             
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   지미는 #Person1# 에게 운동하러 가자고 제안한다. 그들은 금요일에 다리를 할 수 있도록 두 날을 바꾼다.                           
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
-------------------------------------------------

Epoch 5/100
Train Loss: 0.447137
Validation Loss: 0.525816, Rouge-1: 0.372769, Rouge-2: 0.129491, Rouge-l: 0.351387
Average ROUGE: 0.284549
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 숨쉬기가 좀 힘들다고 #Person2# 에게 말합니다. #Person1# 는 알레르기가 없고 운동을 할 때 많이 나타납니다.                             
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   지미는 #Person1# 에게 운동하러 가자고 제안한다. 그들은 금요일에 다리를 할 수 있도록 두 날을 바꾸기로 한다.                               
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
----------------------------------------------------

Epoch 6/100
Train Loss: 0.412844
Validation Loss: 0.533887, Rouge-1: 0.370534, Rouge-2: 0.128973, Rouge-l: 0.348303
Average ROUGE: 0.282603
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 #Person2# 에게 최근 숨쉬기가 힘들다고 말하고, 알레르기가 없다고 말한다. #Person1# 는 폐 전문의에게 천식 검사를 받도록 요청할 것이다.                          
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 지미에게 운동하러 가자고 제안한다. 지미는 팔과 배를 운동하자고 제안하지만, #Person2# 은 주간 스케줄을 따르고 있다. 그들은 금요일에 다리를 할 것이다.                      
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
-------------

In [ ]:
def predict(tokenizer, model, device, test_loader, fname):
    model.eval()
    summary = []
    with torch.no_grad():
        for input_ids in tqdm(test_loader):
            input_ids = input_ids.to(device, dtype=torch.long)

            pred_ids = model.generate(
                input_ids=input_ids,
                max_length=256, 
                num_beams=4,
                repetition_penalty=2.0, 
                length_penalty=1.0, 
                early_stopping=True,
                no_repeat_ngram_size=2
            )
            for ids in pred_ids:
                result = tokenizer.decode(ids)
                summary.append(result)
                
    remove_tokens = ['<usr>', f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": fname,
            "summary" : preprocessed_summary,
        }
    )
    return output

In [ ]:
best_model = torch.load(f'{save_path}/weights/best.pth')
output = predict(tokenizer, model, device, test_loader, test_df['fname'])

In [ ]:
output.to_csv(f"{save_path}/prediction.csv", index=False)